In [1]:
import requests, time, json, math
from tqdm.notebook import tqdm
import threading

#### Retrieve dataset names

In [67]:
dataset_names = requests.get("https://data.gov.sg/api/action/package_list")
if dataset_names.status_code == 200:
    dataset_names = dataset_names.json()["result"]
    print(f'{len(dataset_names)} dataset names retrieved')
else:
    print("Failed to retrieve dataset names")

1860 dataset names retrieved


#### Retrieve dataset details

In [69]:
def retrieve_and_store_dataset_details(dataset_names):
    for dataset_name in tqdm(dataset_names):
        num_tries, status_code = 0, -1
        # Retrieve details
        while status_code != 200 and num_tries<3:
            time.sleep(num_tries*2)
            curr_details = requests.get("https://data.gov.sg/api/action/package_show", params={"id":dataset_name})
            status_code = curr_details.status_code
            num_tries += 1
        # Check success of retrieval
        if curr_details.status_code == 200:
            dataset_details[dataset_name] = curr_details.json()
        else:
            print(f"{dataset_name} details retrieval failed")
            failed_dataset_retrievals.append(dataset_name)

In [70]:
# Retrieve dataset details
dataset_details = {dataset_name:{} for dataset_name in dataset_names}
failed_dataset_retrievals = []

api_calls_per_thread = 50
num_threads_required = math.ceil(len(dataset_names)/api_calls_per_thread)

threads = []
for thread_idx in range(num_threads_required):
    curr_thread = threading.Thread(
        target = retrieve_and_store_dataset_details, 
        args = (dataset_names[thread_idx*api_calls_per_thread : (thread_idx+1)*api_calls_per_thread],)
    )
    curr_thread.start()
    threads.append(curr_thread)
for curr_thread in threads:
    curr_thread.join()

In [71]:
failed_dataset_retrievals

[]

In [72]:
for dataset_name in failed_dataset_retrievals:
    del dataset_details[dataset_name]
len(dataset_details)

1860

In [73]:
with open("data/dataset_details.json", "w") as f:
    json.dump(dataset_details, f)